In [ ]:
import json
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

In [ ]:
#pip install -q "qdrant-client[fastembed]>=1.14.2"
#pip install fastembed

Note: you may need to restart the kernel to use updated packages.


In [5]:
with open('/workspaces/LLM_Zoomcamp/file_documents/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)


documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
qd_client = QdrantClient("http://localhost:6333")

In [ ]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"
collection_name = "zommcamp-faq"

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema='keyword'
)

points = []

for i, doc in enumerate(documents):
    text = f"{doc['question']} {doc['text']}"
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

print(points[0])

True

In [23]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  3.67it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
def vector_search(query, course, limit ,r_type=None):  
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit,
        with_payload=True 
    )

    if r_type == 'score':
        score = []
        for points in query_points.points:
            score.append(points.score)
        return score
    else : 
        result = []

        for points in query_points.points:
            result.append(points.payload)
            
        return result

In [52]:
query = "How can i join this course"

limit = 5
course = "data-engineering-zoomcamp"
dd = vector_search(query=query,course=course,limit=limit)
print(dd)

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}, {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.', 'section': 'General course-related questions', 'question': 'How can we contribute to the course?', 'course': 'data-engineering-zoomcamp'}, {'text': 'After you create a GitHub account, you should clone the course repo to your